In [1]:
import math
from interval import interval, imath
from Polynomials import *

In [2]:
class Taylor:
    def __init__(self, poly:Poly, x0:interval, domain:interval):
        self.poly=poly
        self.rem=interval(0)
        self.x0=x0
        self.I=domain

    def __repr__(self):
        return 'TaylorModel({0}, {1}, {2}, {3} )'.format(self.poly, self.rem, self.x0, self.I)
    
    #bound polynomials
    def bound_poly(self):
        B=interval(0)
        n=self.poly.order
        for i in range(n,-1,-1):
            B=B*(self.I-self.x0)+interval(self.poly.coef[i])
        return B
    
    #bound remainder
    def bound_rem(self, f, selfI):
        order=self.poly.order+1
        if (selfI[0].sup<0 or selfI[0].inf>0):
            a=interval(self.I[0].inf)
            b=interval(self.I[0].sup)
            deltainf=f(a)-self.poly(a-self.x0)
            deltasup=f(b)-self.poly(b-self.x0)
            deltax0=f(self.x0)-self.poly(0)
            delta=interval.hull((interval.hull((deltainf,deltax0)),deltasup))
        else:
            delta=selfI*((self.I-self.x0)**order)
        return delta
        
    def bound(self):
        return bound_poly(self)+self.rem
    
    

In [3]:
a=Taylor(Poly([0,1,2,3,4],4),interval(0),interval([-1,1]))
a1=Poly([0,1,2,3,4],4)
a

TaylorModel(Poly([0, 1, 2, 3, 4], 4), interval([0.0]), interval([0.0]), interval([-1.0, 1.0]) )

In [4]:
a.bound_poly()

interval([-10.0, 10.0])

In [5]:
b=Poly([0,1,0,0,0],2)
I=interval[-0.5,0.5]
imath.exp(I)/6*I**3

interval([-0.03434835980625268, 0.03434835980625268])

In [6]:
imath.exp(I)

interval([0.6065306597126333, 1.6487212707001282])

In [7]:
imath.exp(interval(-0.5))-b.exp()(interval(-0.5))

interval([-0.018469340287366687, -0.018469340287366576])

In [8]:
imath.exp(interval(0))-b.exp()(interval(0))

interval([-0.0])

In [9]:
imath.exp(interval(0.5))-b.exp()(interval(0.5))

interval([0.023721270700127972, 0.023721270700128194])

In [10]:
b.exp()

Poly([1.0, 1.0, 0.5], 2)

In [14]:
bI=imath.exp(I)/6
bI

interval([0.10108844328543887, 0.2747868784500214])

In [15]:
b1=Taylor(b.exp(),interval(0),I)
b1

TaylorModel(Poly([1.0, 1.0, 0.5], 2), interval([0.0]), interval([0.0]), interval([-0.5, 0.5]) )

In [16]:
b1.bound_rem(imath.exp,bI)

interval([-0.018469340287366687, 0.023721270700128194])